

## <b>THIS NOTEBOOK AIMS TO EXTRACT TRANSFORM LOAD CENSUS AND CRIME STATISTICAL DATA</b>
***
1. Census  data is obtained from Australian Bureau of statistics.
2. Crime data is obtained from NSW Bureau of Crime Statistics and Research

Census Data Packs were used to obtain [census data](https://www.abs.gov.au/census/find-census-data/datapacks)

The Following options were 

1. Year :- 2021
2. Datapack type :- General Community profile.
3. Geography :- Postal Area ( POA ) And Local Government area (LGA)

### <b>MODULES REQUIRED </b>

In [2]:
import pandas as pd
import glob
import os 
import numpy as np
import re

import configparser
from etl_helper import * 


import psycopg2
import configparser

import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from io import StringIO
import boto3

In [3]:
!pwd

/Users/icmec/Documents/project_census_crime/ETLs


In [4]:
#Global Vars
currentDirectory = '/Users/icmec/Documents/project_census_crime'
currentDirectory

'/Users/icmec/Documents/project_census_crime'

### <b>POA DATA</b>

#### Loading all postal address data tables to postgres 

In [ ]:
postgresConfig = GetConfiguration()

engine = create_engine(f'postgresql://{postgresConfig["user"]}:{postgresConfig["password"]}@{postgresConfig["host"]}:{postgresConfig["port"]}/{postgresConfig["dbname"]}')
Session = sessionmaker(bind=engine)


with Session() as session:
    for each_file in glob.glob(r'<PATH_TO_CSV_CONTAINING_CENSUS_DATA>'):
        df = pd.read_csv(each_file)
        df.replace('..',np.nan, inplace=True)
        fileName = os.path.basename(each_file)
        nameOfDb , _ = os.path.splitext( fileName ) #add code to rename the dbName here 
        dtypes = { each_column : sqlalchemy.types.Integer() for each_column in df.columns[1:] }  
        df.to_sql(nameOfDb, con=engine, schema = 'censusNswInit' , if_exists='replace',index=False , dtype=dtypes)
        #os.remove(each_file)

#### Push dictionary containing cell description information for 2021 GCP data packs 

In [ ]:
pao_dictionary_df = pd.read_excel('/Users/icmec/Downloads/2021_GCP_POA_for_NSW_short-header/Metadata/Metadata_2021_GCP_DataPacks_R1_R2.xlsx' , skiprows=10 , sheet_name='Cell Descriptors Information' )

postgresConf = GetConfiguration()
SendData(pao_dictionary_df,table_name='2021_census.gcp_poa_nsw.dict',  if_exists_flag = 'replace'  )

#### Consolidated POA dataset 

In [ ]:
merged = ExtractTables(path_to_folder = '/Users/icmec/Downloads/2021_GCP_POA_for_NSW_short-header/2021 Census GCP Postal Areas for NSW',join_on ='POA_CODE_2021')
current_directory = os.getcwd()
merged.to_csv(f'{current_directory}/data/2021_census.gcp_all.poa.csv' , index= False)
gcp_all = pd.read_csv('/Users/icmec/Documents/project_census_crime/data/2021_census.gcp_all.poa.csv')
SendData(gcp_all.T,table_name='2021_census.gcp_all.poa',  if_exists_flag = 'replace' , index=True )


### <b>LGA DATA</b>

#### Preparing the LGA data 

In [ ]:
#ExtractTables function joins all the individual  tables together by LGA_CODE_2021
merged_LGA = ExtractTables(path_to_folder = '/Users/icmec/Downloads/2021_GCP_LGA_for_NSW_short-header/2021 Census GCP Local Government Areas for NSW',join_on ='LGA_CODE_2021')

In [ ]:
currentDirectory = os.getcwd()
merged_LGA.to_csv(f'{currentDirectory}/data/2021_census.gcp_all.lga.csv' , index = False )

In [ ]:
lga_census = pd.read_csv('https://data-lake-icmec-au.s3.ap-southeast-2.amazonaws.com/projectCrimeCensus/2021_census.gcp_all.lga.csv')

In [ ]:
lga_census['lga_code_2021'] =  lga_census['lga_code_2021'].str.extract('(\d+)')

###  <b>WORKING ON CREATING A MAPPING DICTIONARY ( FOR COLUMN NAMES )</b>

In [11]:
meta_data = pd.read_excel('/Users/icmec/Downloads/2021_GCP_LGA_for_NSW_short-header/Metadata/Metadata_2021_GCP_DataPacks_R1_R2.xlsx', skiprows=10, sheet_name = 'Cell Descriptors Information'  )
meta_data_copy = meta_data.copy( deep= True)
#meta_data = meta_data_copy.copy( deep= True)


In [29]:

meta_data['Short'] = meta_data['DataPackfile'] + '.' +  meta_data['Short']
meta_data['Short'] = meta_data['Short'].str.lower()


meta_data['Long'] = meta_data['DataPackfile'] + '.' +  meta_data['Long']
meta_data['Long'] = meta_data['Long'].str.lower()

meta_data.set_index('Short', inplace = True)

meta_data.rename( columns= {'Long':'LongName'} , inplace=True )
display(meta_data)

,Sequential,LongName,DataPackfile,Profiletable,Columnheadingdescriptioninprofile
Short,,,,,
g01.tot_p_m,G1,g01.total_persons_males,G01,G01,Males
g01.tot_p_f,G2,g01.total_persons_females,G01,G01,Females
g01.tot_p_p,G3,g01.total_persons_persons,G01,G01,Persons
g01.age_0_4_yr_m,G4,g01.age_groups_0_4_years_males,G01,G01,Males
g01.age_0_4_yr_f,G5,g01.age_groups_0_4_years_females,G01,G01,Females
...,...,...,...,...,...
g62.method_travel_to_work_ns_f,G16980,g62.method_of_travel_to_work_not_stated_females,G62,G62,Females
g62.method_travel_to_work_ns_p,G16981,g62.method_of_travel_to_work_not_stated_persons,G62,G62,Persons
g62.tot_m,G16982,g62.total_males,G62,G62,Males


In [66]:
SendData(meta_data, table_name='2016_census.gcp_nsw.dict_transformed',index=True,schema_name='prod')

'success'

In [41]:
def makeTableNumber(x):
    if( len(x) > 3 ):
        return x[:3]
    else:
         return x

#meta_data.reset_index( inplace=True )
#meta_data.index = meta_data.DataPackfile.apply(makeTableNumber)
meta_data

,Short,Sequential,LongName,DataPackfile,Profiletable,Columnheadingdescriptioninprofile
DataPackfile,,,,,,
G01,g01.tot_p_m,G1,g01.total_persons_males,G01,G01,Males
G01,g01.tot_p_f,G2,g01.total_persons_females,G01,G01,Females
G01,g01.tot_p_p,G3,g01.total_persons_persons,G01,G01,Persons
G01,g01.age_0_4_yr_m,G4,g01.age_groups_0_4_years_males,G01,G01,Males
G01,g01.age_0_4_yr_f,G5,g01.age_groups_0_4_years_females,G01,G01,Females
...,...,...,...,...,...,...
G62,g62.method_travel_to_work_ns_f,G16980,g62.method_of_travel_to_work_not_stated_females,G62,G62,Females
G62,g62.method_travel_to_work_ns_p,G16981,g62.method_of_travel_to_work_not_stated_persons,G62,G62,Persons
G62,g62.tot_m,G16982,g62.total_males,G62,G62,Males


In [25]:
meta_number_name = pd.read_excel('/Users/icmec/Downloads/2021_GCP_LGA_for_NSW_short-header/Metadata/Metadata_2021_GCP_DataPacks_R1_R2.xlsx', skiprows=8, sheet_name = "tableName"  )
# meta_data.reset_index()


In [47]:
#merge
#meta_data_name_merged = meta_data.merge( meta_number_name , left_index=True , right_on='Table Number' , how='inner'  )
#meta_data_name_merged.set_index('Table Number' , inplace=True)
meta_data_name_merged

,Short,Sequential,LongName,DataPackfile,Profiletable,Columnheadingdescriptioninprofile,Table Name,Table population
Table Number,,,,,,,,
G01,g01.tot_p_m,G1,g01.total_persons_males,G01,G01,Males,Selected Person Characteristics by Sex,Persons
G01,g01.tot_p_f,G2,g01.total_persons_females,G01,G01,Females,Selected Person Characteristics by Sex,Persons
G01,g01.tot_p_p,G3,g01.total_persons_persons,G01,G01,Persons,Selected Person Characteristics by Sex,Persons
G01,g01.age_0_4_yr_m,G4,g01.age_groups_0_4_years_males,G01,G01,Males,Selected Person Characteristics by Sex,Persons
G01,g01.age_0_4_yr_f,G5,g01.age_groups_0_4_years_females,G01,G01,Females,Selected Person Characteristics by Sex,Persons
...,...,...,...,...,...,...,...,...
G62,g62.method_travel_to_work_ns_f,G16980,g62.method_of_travel_to_work_not_stated_females,G62,G62,Females,Method of Travel to Work by Sex,Employed persons aged 15 years and over
G62,g62.method_travel_to_work_ns_p,G16981,g62.method_of_travel_to_work_not_stated_persons,G62,G62,Persons,Method of Travel to Work by Sex,Employed persons aged 15 years and over
G62,g62.tot_m,G16982,g62.total_males,G62,G62,Males,Method of Travel to Work by Sex,Employed persons aged 15 years and over


In [48]:
SendData(meta_data_name_merged, table_name='2021_census.gcp_nsw.dict_transformed',schema_name='prod',if_exists_flag='replace',index=True)


'success'

## EXTRACTING CENSUS 2016

In [38]:
def compareLgas(list_2021,list_2016):
    difference_2021_2016 = set(list_2021) - set(list_2016)
    #print( difference_2021_2016)
    difference_2016_2021 = set(list_2016) - set(list_2021)
    #print( difference_2016_2021)
    print( f'Number of items intersecting = {len(set(list_2021).intersection( set(list_2016) ))}'  )
    print( f'Number of items not intersecting = {len(set(list_2021).union( set(list_2016) ))}'  )

compareLgas( lga_2021_Merged['lga_code_2021'], lga_2016_Merged['lga_code_2016'])



Number of items intersecting = 126
Number of items not intersecting = 137


In [39]:
compareLgas(lga_2021_Merged.columns,lga_2016_Merged.columns)

Number of items intersecting = 4454
Number of items not intersecting = 28067


In [52]:
lga_2016_Merged.to_csv('/Users/icmec/Documents/project_census_crime/data/lga/2016_census.gcp_all.lga.csv')

In [9]:
view = GetData('''SELECT * FROM prod."2021_census.gcp_all_juvenile_victims.lga"''')

In [35]:
view.T[[16986]].to_clipboard()

In [47]:
#view = GetData('''SELECT * FROM prod."2021_census.gcp_all_juvenile_victims.lga"''')
#viewTransformed =  view.T
#newHeader = viewTransformed.loc['index',]
#viewTransformed.columns = newHeader
#viewTransformed.drop( 'index' , axis=0 , inplace=True)
#viewTransformed['lga_code_2021'] = viewTransformed['lga_code_2021'].astype('int64')
#viewTransformed.to_csv(f'{currentDirectory}/data/lga/2021_census.gcp_all_juvenile_victims_with_names.lga.csv')

In [59]:
viewTransformed.columns[-11:]


Index(['homicide.rate', 'domestic_violence_related_assault.rate',
       'nondomestic_violence_related_assault.rate', 'sexual_offences.rate',
       'abduction_and_kidnapping.rate', 'robbery.rate',
       'blackmail_and_extortion.rate',
       'intimidation_stalking_and_harassment.rate',
       'other_offences_against_the_person.rate', 'arson.rate',
       'breach_apprehended_violence_order.rate'],
      dtype='object', name='index')

## Quessland census data

In [5]:
queenslandMerged_LGA = ExtractTables(path_to_folder = '/Users/icmec/Downloads/Queenland /2021_GCP_LGA_for_QLD_short-header/2021 Census GCP Local Government Areas for QLD',join_on ='LGA_CODE_2021')
queenslandMerged_LGA['lga_code_2021'] =  queenslandMerged_LGA['lga_code_2021'].str.extract('(\d+)')


In [17]:
queenslandMerged_LGA

,lga_code_2021,g51b.f_health_occ_mng,g51b.f_health_occ_pro,g51b.f_health_occ_ttw,g51b.f_health_occ_cps,g51b.f_health_occ_ca,g51b.f_health_occ_sal,g51b.f_health_occ_mod,g51b.f_health_occ_lab,g51b.f_health_occ_idns,...,g40.rent_ns_lt_other_lld_type,g40.rent_ns_lt_lld_type_ns,g40.rent_ns_tot,g40.tot_lt_real_este_agent,g40.tot_lt_ste_ter_hsg_auth,g40.tot_lt_com_hou_pro,g40.tot_lt_psn_not_sme_hhd,g40.tot_lt_other_lld_type,g40.tot_lt_lld_type_ns,g40.tot_tot
0,30250,0,0,0,0,0,0,0,0,0,...,0,3,22,0,180,23,0,13,0,218
1,30300,19,82,3,23,7,0,0,0,0,...,30,0,52,169,95,25,133,106,3,534
2,30370,47,130,12,65,31,5,0,10,5,...,52,0,114,555,191,24,437,268,22,1499
3,30410,13,39,5,18,4,0,0,3,0,...,24,0,40,65,40,11,67,84,0,270
4,30450,4,5,0,0,0,0,0,4,0,...,4,0,8,0,4,0,3,25,0,38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,37550,0,4,0,5,6,0,0,0,0,...,3,0,9,0,228,14,0,22,0,265
76,37570,0,0,0,0,0,0,0,0,0,...,0,0,0,0,13,54,0,0,0,73
77,37600,0,6,0,10,3,0,0,0,0,...,0,0,5,0,60,313,0,10,0,382
78,39499,7,107,0,35,13,6,0,15,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
#Saving to S3 
client = boto3.client('s3')
csv_buffer = StringIO()
queenslandMerged_LGA.to_csv(csv_buffer ,  index=False )
client.put_object( Body=csv_buffer.getvalue(), Bucket='data-lake-icmec-au', StorageClass = 'ONEZONE_IA' ,Key='projectCrimeCensus/landingZone/2021_census.gcp_all.lga_qld.csv')

data-lake-icmec-au


In [23]:
qld_census = pd.read_csv('https://data-lake-icmec-au.s3.ap-southeast-2.amazonaws.com/projectCrimeCensus/landingZone/2021_census.gcp_all.lga_qld.csv',index_col = 0 )
qld_census

,g51b.f_health_occ_mng,g51b.f_health_occ_pro,g51b.f_health_occ_ttw,g51b.f_health_occ_cps,g51b.f_health_occ_ca,g51b.f_health_occ_sal,g51b.f_health_occ_mod,g51b.f_health_occ_lab,g51b.f_health_occ_idns,g51b.f_health_total,...,g40.rent_ns_lt_other_lld_type,g40.rent_ns_lt_lld_type_ns,g40.rent_ns_tot,g40.tot_lt_real_este_agent,g40.tot_lt_ste_ter_hsg_auth,g40.tot_lt_com_hou_pro,g40.tot_lt_psn_not_sme_hhd,g40.tot_lt_other_lld_type,g40.tot_lt_lld_type_ns,g40.tot_tot
lga_code_2021,,,,,,,,,,,,,,,,,,,,,
30250,0,0,0,0,0,0,0,0,0,0,...,0,3,22,0,180,23,0,13,0,218
30300,19,82,3,23,7,0,0,0,0,130,...,30,0,52,169,95,25,133,106,3,534
30370,47,130,12,65,31,5,0,10,5,301,...,52,0,114,555,191,24,437,268,22,1499
30410,13,39,5,18,4,0,0,3,0,83,...,24,0,40,65,40,11,67,84,0,270
30450,4,5,0,0,0,0,0,4,0,9,...,4,0,8,0,4,0,3,25,0,38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37550,0,4,0,5,6,0,0,0,0,15,...,3,0,9,0,228,14,0,22,0,265
37570,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,13,54,0,0,0,73
37600,0,6,0,10,3,0,0,0,0,22,...,0,0,5,0,60,313,0,10,0,382
